develop the add existing base years per grade

In [ ]:
%load_ext autoreload
%autoreload 2
%precision %e

In [ ]:
# running the jupyter notebook on the compute nodes doesnt build the path as expected, you have to manually do this
import IPython, os

working_directory = os.path.dirname(IPython.extract_module_locals()[1]['__vsc_ipynb_file__'])
workflow_dir = os.path.dirname(working_directory)
scripts_dir = os.path.join(workflow_dir, "scripts")
root_dir = os.path.dirname(workflow_dir)

os.chdir(scripts_dir)


In [ ]:
import logging
import pypsa
import os.path
from _helpers import mock_snakemake
from _pypsa_helpers import shift_profile_to_planning_year

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

In [ ]:
target = os.path.join(root_dir, "resources/derived_data/cutout_China-2020c/solar_regions_by_class_min_cf_delta0.02_n2.geojson")
spv = gpd.read_file(target)
wind = gpd.read_file(os.path.join(root_dir, "resources/derived_data/cutout_China-2020c/onwind_regions_by_class_min_cf_delta0.07_n3.geojson"))


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 2,figsize=(12, 5))
# Adjust colorbar to fit the subplot height

divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=-0.3)
spv.plot("cf", cmap="magma_r", legend=True, edgecolor=None, linewidth=2, ax=ax[0], cax=cax)
spv.dissolve("bus").boundary.plot(edgecolor="white", facecolor="none", linewidth=0.7, ax=ax[0])
ax[0].set_title("Solar PV Availability Factor")

divider2 = make_axes_locatable(ax[1])
cax2 = divider2.append_axes("right", size="5%", pad=-0.3)

wind[wind.cf>0.02].plot(
    "cf",
    cmap="magma_r",
    legend=True,
    edgecolor=None,
    linewidth=2,
    ax=ax[1],
    cax=cax2,
    vmin=0.05,
    missing_kwds={"color": "white"}
)
wind.dissolve("bus").boundary.plot(edgecolor="black", facecolor="none", linewidth=0.7, ax=ax[1])
ax[1].set_title("Wind Availability Factor")
fig.tight_layout()

In [ ]:
if "snakemake" not in globals():
    snakemake = mock_snakemake(
        "add_existing_baseyear",
        topology="current+FCG",
        snakefile_path=os.path.abspath("../"),
        co2_pathway="remind_ssp2NPI",
        planning_horizons="2025",
        heating_demand="positive",
    )

In [ ]:
import numpy as np
from add_electricity import load_costs

In [ ]:
from add_existing_baseyear import read_existing_capacities, fix_existing_capacities, assign_year_bins
from add_existing_baseyear import add_power_capacities_installed_before_baseyear, add_existing_vre_capacities

In [ ]:
tech_costs = snakemake.input.tech_costs
config = snakemake.config
baseyear = snakemake.params["baseyear"]
cost_year = snakemake.wildcards["planning_horizons"]

n = pypsa.Network(snakemake.input.network)
n_years = n.snapshot_weightings.generators.sum() / 8760

data_paths = {k: v for k, v in snakemake.input.items()}
costs = load_costs(tech_costs, config["costs"], config["electricity"], cost_year, n_years)


In [ ]:
baseyear = snakemake.params["baseyear"]
# add_build_year_to_new_assets(n, baseyear)
if snakemake.params["add_baseyear_to_assets"]:
    add_build_year_to_new_assets(n, baseyear)

config = snakemake.config
tech_costs = snakemake.input.tech_costs
cost_year = snakemake.wildcards["planning_horizons"]
data_paths = {k: v for k, v in snakemake.input.items()}
vre_techs = ["solar", "onwind", "offwind"]

costs = load_costs(tech_costs, config["costs"], config["electricity"], cost_year, n_years)

existing_capacities = read_existing_capacities(data_paths)
year_bins = config["existing_capacities"]["grouping_years"]
# TODO add renewables
existing_capacities = assign_year_bins(existing_capacities, year_bins)
df = fix_existing_capacities(existing_capacities, costs, year_bins, baseyear)

vre_caps = df.query("Tech in @vre_techs | Fueltype in @vre_techs")
# vre_caps.loc[:, "Country"] = coco.CountryConverter().convert(["China"], to="iso2")
vres = add_existing_vre_capacities(n, costs, vre_caps, config)
df = pd.concat([df.query("Tech not in @vre_techs & Fueltype not in @vre_techs"), vres], axis=0)


In [ ]:
df.resource_class.fillna("none", inplace=True)
df.pivot_table(
        index=["grouping_year", "Fueltype", "resource_class"],
        columns="bus",
        values="Capacity",
        aggfunc="sum",
    )
stored = df.copy

In [ ]:
stored.renamed)

In [ ]:
capacity.index.values

In [ ]:
carrier = {
    "coal": "coal power plant",
    "CHP coal": "CHP coal",
    "CHP gas": "CHP gas",
    "OCGT": "OCGT gas",
    "solar": "solar",
    "solar thermal": "solar thermal",
    "onwind": "onwind",
    "offwind": "offwind",
    "coal boiler": "coal boiler",
    "ground heat pump": "heat pump",
    "nuclear": "nuclear",
}

# in case user forgot to do it
df = fix_existing_capacities(
    df, costs, config["existing_capacities"]["grouping_years"], baseyear
)
df.resource_class.fillna("none", inplace=True)
df = df.pivot_table(
    index=["grouping_year", "Fueltype", "resource_class"],
    columns="bus",
    values="Capacity",
    aggfunc="sum",
)
# TODO do we really need to loop over the years?
for grouping_year, generator, resouce_class in df.index:
    print(generator, grouping_year)
    # capacity is the capacity in MW at each node for this
    capacity = df.loc[grouping_year, generator]
    capacity = capacity[~capacity.isna()]
    capacity = capacity[capacity > config["existing_capacities"]["threshold_capacity"]].T
    capacity.fillna(0, inplace=True)
    if capacity.values.max() == 0:
        continue
    
    vre_carriers = ["solar", "onwind", "offwind"]
    if generator in vre_carriers:
        mask = n.generators_t.p_max_pu.columns.map(n.generators.carrier) == generator
        p_max_pu = n.generators_t.p_max_pu.loc[:, mask]
        n.add(
            "Generator",
            capacity.index,
            suffix=" " + generator + "-" + str(grouping_year),
            bus=capacity.index.values,
            carrier=carrier[generator],
            p_nom=capacity,
            p_nom_min=capacity,
            p_nom_extendable=False,
            marginal_cost=costs.at[generator, "marginal_cost"],
            capital_cost=costs.at[generator, "capital_cost"],
            efficiency=costs.at[generator, "efficiency"],
            p_max_pu=p_max_pu.rename(columns=n.generators.bus),
            build_year=grouping_year,
            lifetime=costs.at[generator, "lifetime"],
        )

    if generator == "coal":
        n.add(
            "Generator",
            capacity.index,
            suffix=" " + generator + "-" + str(grouping_year),
            bus=capacity.index.values,
            carrier=carrier[generator],
            p_nom=capacity,
            p_nom_extendable=False,
            marginal_cost=costs.at[generator, "marginal_cost"],
            capital_cost=costs.at[generator, "capital_cost"],
            efficiency=costs.at[generator, "efficiency"],
            build_year=grouping_year,
            lifetime=costs.at[generator, "lifetime"],
        )

    if generator == "nuclear":
        n.add(
            "Generator",
            capacity.index,
            suffix=" " + generator + "-" + str(grouping_year),
            bus=capacity.index.values,
            carrier=carrier[generator],
            p_nom=capacity,
            p_nom_min=capacity,
            p_nom_extendable=False,
            p_min_pu=0.7,
            marginal_cost=costs.at[generator, "marginal_cost"],
            capital_cost=costs.at[generator, "capital_cost"],
            efficiency=costs.at[generator, "efficiency"],
            build_year=grouping_year,
            lifetime=costs.at[generator, "lifetime"],
        )

    if generator == "solar thermal" and config["heat_coupling"]:
        p_max_pu = n.generators_t.p_max_pu[capacity.index + " central " + generator]
        p_max_pu.columns = capacity.index
        n.add(
            "Generator",
            capacity.index,
            suffix=" central " + generator + "-" + str(grouping_year),
            bus=capacity.index.values + " central heat",
            carrier=carrier[generator],
            p_nom=capacity,
            p_nom_min=capacity,
            p_nom_extendable=False,
            marginal_cost=costs.at["central " + generator, "marginal_cost"],
            capital_cost=costs.at["central " + generator, "capital_cost"],
            p_max_pu=p_max_pu,
            build_year=grouping_year,
            lifetime=costs.at["central " + generator, "lifetime"],
        )

    if generator == "CHP coal" and config["heat_coupling"]:
        bus0 = capacity.index + " coal"
        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + " generator" + "-" + str(grouping_year),
            bus0=bus0,
            bus1=capacity.index,
            carrier=carrier[generator],
            marginal_cost=0.37 * costs.at["central coal CHP", "VOM"],  # NB: VOM is per MWel
            capital_cost=0.37
            * costs.at["central coal CHP", "capital_cost"],  # NB: fixed cost is per MWel,
            p_nom=capacity / 0.37,
            p_nom_min=capacity / 0.37,
            p_nom_extendable=False,
            efficiency=0.37,
            p_nom_ratio=1.0,
            c_b=0.75,
            build_year=grouping_year,
            lifetime=costs.at["central coal CHP", "lifetime"],
        )

        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + " boiler" + "-" + str(grouping_year),
            bus0=bus0,
            bus1=capacity.index + " central heat",
            carrier=carrier[generator],
            marginal_cost=0.37 * costs.at["central coal CHP", "VOM"],  # NB: VOM is per MWel
            p_nom=capacity / 0.37 * 0.15,
            p_nom_min=capacity / 0.37 * 0.15,
            p_nom_extendable=False,
            efficiency=0.37 / 0.15,
            build_year=grouping_year,
            lifetime=costs.at["central coal CHP", "lifetime"],
        )

    if generator == "CHP gas" and config["heat_coupling"]:
        bus0 = capacity.index + " gas"
        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + " generator" + "-" + str(grouping_year),
            bus0=bus0,
            bus1=capacity.index,
            carrier=carrier[generator],
            marginal_cost=costs.at["central gas CHP", "efficiency"]
            * costs.at["central gas CHP", "VOM"],  # NB: VOM is per MWel
            capital_cost=costs.at["central gas CHP", "efficiency"]
            * costs.at["central gas CHP", "capital_cost"],  # NB: fixed cost is per MWel,
            p_nom=capacity / costs.at["central gas CHP", "efficiency"],
            p_nom_min=capacity / costs.at["central gas CHP", "efficiency"],
            p_nom_extendable=False,
            efficiency=costs.at["central gas CHP", "efficiency"],
            p_nom_ratio=1.0,
            c_b=costs.at["central gas CHP", "c_b"],
            build_year=grouping_year,
            lifetime=costs.at["central gas CHP", "lifetime"],
        )
        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + " boiler" + "-" + str(grouping_year),
            bus0=bus0,
            bus1=capacity.index + " central heat",
            carrier=carrier[generator],
            marginal_cost=costs.at["central gas CHP", "efficiency"]
            * costs.at["central gas CHP", "VOM"],  # NB: VOM is per MWel
            p_nom=capacity
            / costs.at["central gas CHP", "efficiency"]
            * costs.at["central gas CHP", "c_v"],
            p_nom_min=capacity
            / costs.at["central gas CHP", "efficiency"]
            * costs.at["central gas CHP", "c_v"],
            p_nom_extendable=False,
            efficiency=costs.at["central gas CHP", "efficiency"]
            / costs.at["central gas CHP", "c_v"],
            build_year=grouping_year,
            lifetime=costs.at["central gas CHP", "lifetime"],
        )

    if generator == "OCGT":
        bus0 = capacity.index + " gas"
        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + "-" + str(grouping_year),
            bus0=bus0,
            bus1=capacity.index,
            carrier=carrier[generator],
            marginal_cost=costs.at[generator, "efficiency"]
            * costs.at[generator, "VOM"],  # NB: VOM is per MWel
            capital_cost=costs.at[generator, "efficiency"]
            * costs.at[generator, "capital_cost"],
            # NB: fixed cost is per MWel
            p_nom=capacity / costs.at[generator, "efficiency"],
            p_nom_min=capacity / costs.at[generator, "efficiency"],
            p_nom_extendable=False,
            efficiency=costs.at[generator, "efficiency"],
            build_year=grouping_year,
            lifetime=costs.at[generator, "lifetime"],
        )

    if generator == "coal boiler" and config["heat_coupling"]:
        bus0 = capacity.index + " coal"
        for cat in [" central "]:
            n.add(
                "Link",
                capacity.index,
                suffix="" + cat + generator + "-" + str(grouping_year),
                bus0=bus0,
                bus1=capacity.index + cat + "heat",
                carrier=carrier[generator],
                marginal_cost=costs.at[cat.lstrip() + generator, "efficiency"]
                * costs.at[cat.lstrip() + generator, "VOM"],
                capital_cost=costs.at[cat.lstrip() + generator, "efficiency"]
                * costs.at[cat.lstrip() + generator, "capital_cost"],
                p_nom=capacity / costs.at[cat.lstrip() + generator, "efficiency"],
                p_nom_min=capacity / costs.at[cat.lstrip() + generator, "efficiency"],
                p_nom_extendable=False,
                efficiency=costs.at[cat.lstrip() + generator, "efficiency"],
                build_year=grouping_year,
                lifetime=costs.at[cat.lstrip() + generator, "lifetime"],
            )
    # TODO fix centralise
    if generator == "ground heat pump" and config["heat_coupling"]:
        date_range = pd.date_range(
            "2025-01-01 00:00",
            "2025-12-31 23:00",
            freq=config["snapshots"]["freq"],
            tz="Asia/shanghai",
        )
        date_range = date_range.map(lambda t: t.replace(year=2020))

        with pd.HDFStore(snakemake.input.cop_name, mode="r") as store:
            gshp_cop = store["gshp_cop_profiles"]
            gshp_cop.index = gshp_cop.index.tz_localize(None)
            gshp_cop = shift_profile_to_planning_year(
                gshp_cop, snakemake.wildcards.planning_horizons
            )
            gshp_cop = gshp_cop.loc[n.snapshots]
        n.add(
            "Link",
            capacity.index,
            suffix=" " + generator + "-" + str(grouping_year),
            bus0=capacity.index,
            bus1=capacity.index + " central heat",
            carrier="heat pump",
            efficiency=(
                gshp_cop[capacity.index]
                if config["time_dep_hp_cop"]
                else costs.at["decentral ground-sourced heat pump", "efficiency"]
            ),
            capital_cost=costs.at["decentral ground-sourced heat pump", "efficiency"]
            * costs.at["decentral ground-sourced heat pump", "capital_cost"],
            marginal_cost=costs.at["decentral ground-sourced heat pump", "efficiency"]
            * costs.at["decentral ground-sourced heat pump", "marginal_cost"],
            p_nom=capacity / costs.at["decentral ground-sourced heat pump", "efficiency"],
            p_nom_min=capacity / costs.at["decentral ground-sourced heat pump", "efficiency"],
            p_nom_extendable=False,
            build_year=grouping_year,
            lifetime=costs.at["decentral ground-sourced heat pump", "lifetime"],
        )

In [ ]:

        if generator == "nuclear":
            n.add(
                "Generator",
                capacity.index,
                suffix=" " + generator + "-" + str(grouping_year),
                bus=capacity.index,
                carrier=carrier[generator],
                p_nom=capacity,
                p_nom_min=capacity,
                p_nom_extendable=False,
                p_min_pu=0.7,
                marginal_cost=costs.at[generator, "marginal_cost"],
                capital_cost=costs.at[generator, "capital_cost"],
                efficiency=costs.at[generator, "efficiency"],
                build_year=grouping_year,
                lifetime=costs.at[generator, "lifetime"],
            )


In [ ]:
n.generators

In [ ]:
out_ = add_power_capacities_installed_before_baseyear(n, costs, config, df)
out_


In [ ]:
grouping_year

In [ ]:
grouping_year

In [ ]:
out = n.generators.groupby(["carrier","bus"]).p_nom_min.sum().sort_values(ascending=False)

In [ ]:
out.loc[("coal", slice(None))]

In [ ]:

for grouping_year, generator, resource_class in df.index:
    # capacity is the capacity in MW at each node for this
    capacity = df.loc[grouping_year, generator, resource_class]
    capacity = capacity[~capacity.isna()]
    capacity = capacity[capacity > 1]
    suffix = "-ac" if generator == "offwind" else ""
    name_suffix = f" {generator}{suffix}-{grouping_year}"

In [ ]:
import xarray as xr
p = "/home/ivanra/documents/PyPSA-China-PIK/resources/derived_data/cutout_China-2020c/profile_onwind-min_cf_delta0.05_n3.nc"
with xr.open_dataset(p) as ds:
    if ds.indexes["bus"].empty:
        pass
    if "year" in ds.indexes:
        ds = ds.sel(year=ds.year.min(), drop=True)

In [ ]:
ds.p_nom_max

In [ ]:
import powerplantmatching as pm
renw = pm.data.IRENASTAT()
renw.query("Country == 'China' & Technology == 'Onshore' & Year == @renw.Year.max()")
renw.head(1)

In [ ]:
import country_converter as coco

In [ ]:
vre_techs = ['onwind',
 'offwind',
 'offwind-ac',
 'offwind-dc',
 'solar',]
vre_caps = existing_capacities.query("Tech in @vre_techs | Fueltype in @vre_techs")
vre_caps.loc[:, "Country"] = coco.CountryConverter().convert(["China"], to="iso2")


In [ ]:
irena = pm.data.IRENASTAT().powerplant.convert_country_to_alpha2()
irena

In [ ]:
from functions import cartesian
import re

In [ ]:
vre_caps.groupby("Tech").Capacity.sum().sort_values(ascending=False)

In [ ]:
grouped = vre_caps.groupby(["Country", "cluster_bus", "Tech", "DateIn"]).Capacity.sum()
vre_caps.Country.unique()

In [ ]:
grouped.unstack().reset_index().query("Country == 'CN'").drop(columns=["Country"])

In [ ]:
grouped_vre = vre_caps.groupby(["Tech", "cluster_bus", "DateIn"]).Capacity.sum()
vre_df = grouped_vre.unstack().reset_index()

# Create a DataFrame with all combinations of cluster_bus and Tech
all_combinations = pd.MultiIndex.from_product(
    [vre_df['cluster_bus'].unique(), vre_df['Tech'].unique()],
    names=['cluster_bus', 'Tech']
).to_frame(index=False)

# Merge with the existing vre_df to fill missing combinations
vre_df = all_combinations.merge(vre_df, on=['cluster_bus', 'Tech'], how='left')

# Fill missing values with NaN or appropriate defaults
vre_df.fillna(0, inplace=True)

vre_df

In [ ]:
df_agg.query(' bus in ["Fujian", "Jiangsu "] & Fueltype=="offwind"').Capacity.sum()

In [ ]:
existing_capacities.query(' cluster_bus in ["Fujian", "Jiangsu "] & Fueltype=="offwind" & DateIn==2020').Capacity.sum()

In [ ]:
tech_map = {"solar": "PV", "onwind": "Onshore", "offwind-ac": "Offshore", "offwind": "Offshore"}
tech_map = {k:tech_map[k] for k in tech_map if k in config["Techs"]["vre_techs"]}
df_vres= pd.DataFrame()

In [ ]:
for country, group in carrier_gens.groupby(
    carrier_gens.bus.map(n.buses.country)):
    break

In [ ]:
is_complete = set(carrier_gens.bus.values).difference(set(nodes))
is_complete

In [ ]:
existing_capacities

In [ ]:
sorted = df_agg.groupby(["Fueltype", "bus"]).Capacity.sum()
sorted

In [ ]:
existing_capacities.query("Fueltype in ['solar', 'offwind', 'onwind']").rename(columns={"cluster_bus": "bus"}).groupby(["Fueltype", "bus"]).Capacity.sum()

In [ ]:
df_agg.groupby("Fueltype").Capacity.sum().sort_values(ascending=False)

In [ ]:
df_ = existing_capacities.groupby(["Fueltype", "DateIn"]).Capacity.sum().sort_values(ascending=False)
df_.loc[(slice(None), 2020)]

In [ ]:
df.query("Country == 'China'").T

In [ ]:
gen_i = n.generators.query("carrier == @carrier").index
carrier_gens = n.generators.loc[gen_i]

In [ ]:
carrier_gens

In [ ]:

existing_capacities

In [ ]:
df_agg.Tech.map(costs.lifetime)

In [ ]:
import numpy as np

availability = np.array([1000, 2000, 500])
to_distribute = 1200

# Cumulative sum tells us how much total would be used up to each point
cumsum = np.cumsum(availability)
# Mask for where the cumulative sum is still below the total to distribute
used_up = cumsum < to_distribute
# First point where we cross the distribution limit
cutoff_index = np.argmax(cumsum >= to_distribute)

# Start with zero allocation
allocation = np.zeros_like(availability)

# Fully allocate where availability is used up
allocation[used_up] = availability[used_up]
# Allocate the remainder at the cutoff index
if to_distribute > cumsum[cutoff_index - 1] if cutoff_index > 0 else 0:
    previous_cumsum = cumsum[cutoff_index - 1] if cutoff_index > 0 else 0
    allocation[cutoff_index] = to_distribute - previous_cumsum

print(allocation)


In [ ]:
availability = np.array([1000, 2000, 500])
to_distribute = np.array([1200, 1900])
n_years = len(to_distribute)
n_sources = len(availability)

# To store allocation per year per source (shape: sources x years)
allocation = np.zeros((n_sources, n_years), dtype=int)
remaining = availability.copy()

for j in range(n_years):
    needed = to_distribute[j]
    cumsum = np.cumsum(remaining)
    used_up = cumsum < needed
    cutoff = np.argmax(cumsum >= needed)

    allocation[used_up, j] = remaining[used_up]

    if needed > (cumsum[cutoff - 1] if cutoff > 0 else 0):
        allocation[cutoff, j] = needed - (cumsum[cutoff - 1] if cutoff > 0 else 0)

    # Subtract what was used from availability
    remaining -= allocation[:, j]

allocation

In [ ]:
if "DateOut" not in existing_capacities.columns:
    existing_capacities["DateOut"] = np.nan
lifetimes = existing_capacities.Fueltype.map(costs.lifetime).fillna(df_agg.Tech.map(costs.lifetime))
existing_capacities.loc[:,"DateOut"] = existing_capacities.DateOut.fillna(lifetimes)

In [ ]:
existing_capacities[existing_capacities.DateOut.isna()]

In [ ]:
df_agg = existing_capacities.copy()
lifetimes = df_agg.Fueltype.map(costs.lifetime).fillna(df_agg.Tech.map(costs.lifetime)).dropna()

df_agg.loc[:, "DateOut"] = df_agg["DateIn"].astype(int) + lifetimes
df_agg

In [ ]:
df_agg = existing_capacities.copy()
# Fill missing DateOut
dateout = df_agg.loc[biomass_i, "DateIn"] + lifetime_values["lifetime"]
df_agg.loc[biomass_i, "DateOut"] = df_agg.loc[biomass_i, "DateOut"].fillna(dateout)

# include renewables in df_agg
add_existing_renewables(
    df_agg=df_agg,
    costs=costs,
    n=n,
    countries=countries,
)
# drop assets which are already phased out / decommissioned
phased_out = df_agg[df_agg["DateOut"] < baseyear].index
df_agg.drop(phased_out, inplace=True)

newer_assets = (df_agg.DateIn > max(grouping_years)).sum()
if newer_assets:
    logger.warning(
        f"There are {newer_assets} assets with build year "
        f"after last power grouping year {max(grouping_years)}. "
        "These assets are dropped and not considered."
        "Consider to redefine the grouping years to keep them."
    )
    to_drop = df_agg[df_agg.DateIn > max(grouping_years)].index
    df_agg.drop(to_drop, inplace=True)

df_agg["grouping_year"] = np.take(
    grouping_years, np.digitize(df_agg.DateIn, grouping_years, right=True)
)

In [ ]:
ppl = pm.powerplants(url_fetch=True)

In [ ]:
from functions import cartesian

In [ ]:
for c, grp in carrier_gens.groupby(carrier_gens.bus.map(n.buses.country)):
    print(c)

In [ ]:
carrier_gens = n.generators.loc[gen_i]
res_capacities = []
for country, group in carrier_gens.groupby(
    carrier_gens.bus.map(n.buses.country)
):
    fraction = group.p_nom_max / group.p_nom_max.sum()
    res_capacities.append(cartesian(irena, fraction))
res_capacities = pd.concat(res_capacities, axis=1).T

In [ ]:
import re
for year in res_capacities.columns:
    for gen in res_capacities.index:
        bus_bin = re.sub(f" {carrier}.*", "", gen)
        bus, bin_id = bus_bin.rsplit(" ", maxsplit=1)
        name = f"{bus_bin} {carrier}-{year}"
        capacity = res_capacities.loc[gen, year]
        if capacity > 0.0:
            cost_key = carrier.split("-", maxsplit=1)[0]
            df_agg.at[name, "Fueltype"] = carrier
            df_agg.at[name, "Capacity"] = capacity
            df_agg.at[name, "DateIn"] = year
            df_agg.at[name, "lifetime"] = costs.at[cost_key, "lifetime"]
            df_agg.at[name, "DateOut"] = (
                year + costs.at[cost_key, "lifetime"] - 1
            )
            df_agg.at[name, "bus"] = bus
            df_agg.at[name, "resource_class"] = bin_id


In [ ]:
df_agg